In [7]:
import pygame, sys, random
import numpy as np
import math
import matplotlib.pyplot as plt
import pylab
from fractions import Fraction

dem_episodes = 250

obstacleRadius = 10
agentRadius = 10
goalRadius = 10

# get size of state and action from environment

dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] - 100, 500]#dispSize[1]]
initPosAgent = initPosAgentStandard#[boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
initPosGoal = [100, 500]

moveObstacles = True
action_size = 8
obsNumber = 200

In [8]:
def stateGenerator(intenPref, distPref):
    returnSum = [intenPref, distPref]
    returnSum = np.reshape(returnSum, [1, state_size])
#     print returnSum
    return returnSum

In [9]:
def takeAction(action):
    xAction = 0
    yAction = 0
    actionOffset = 1
    if action == 0:
        xAction = actionOffset
    elif action == 1:
        xAction = actionOffset
        yAction = actionOffset
    elif action == 2:
        xAction = actionOffset
        yAction = -actionOffset
    elif action == 3:
        xAction = -actionOffset
        yAction = actionOffset
    elif action == 4:
        xAction = -actionOffset
    elif action == 5:
        xAction = -actionOffset
        yAction = -actionOffset
    elif action == 6:
        yAction = -actionOffset
    elif action == 7:
        yAction = actionOffset
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [10]:
def makeFeature(agtPos, goalPos, obsPos, const):
    feature1 = (agtPos[0] - goalPos[0])**2 + (agtPos[1] - goalPos[1])**2
    feature2 = 1 / ((agtPos[0] - obsPos[0])**2 + (agtPos[1] - obsPos[1])**2 + const)
    return [feature1, feature2]

In [11]:
def rangeFinder(allObsPos, rangeCenter):
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
    index = np.argmin(allObsAgtDistance)
    return index

In [12]:
pygame.init()

screen = pygame.display.set_mode(dispSize)
screen.fill([200, 200, 200])

rList, episodes = [], []
yOffset = 100
xOffset = 50
# weightVec = [-0.23, -0.1696] # PEARL Parameter
# weightVec = [-0.85232079, -0.8940559] #Trained Parameter from [90, 90]
weightVec = [-0.08357393, -0.6748898]
# weightVec = [-0.88452816, -0.64293331]
# weightVec = [-0.00157787, -0.75601858]
# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
# for i in range(0,obsNumber):
#     obstaclePos[i][0] = int(150 + i % 15 * 100)
#     obstaclePos[i][1] = int(yOffset + i / 15 * 100)
for i in range(0,obsNumber):
    obstaclePos[i][0] = int(150 + i % 14 * 80)
    obstaclePos[i][1] = int(80 + i / 14 * 80)

Qs = [0 for _ in range(action_size)]
for e in range(dem_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    print("Episode ", e, "Starts!")
    xMove = 0
    yMove = 0
    while not done:

        for i in range(0,action_size):
            [tmpXMove, tmpYMove] = takeAction(i)
            tmpX = x + tmpXMove
            tmpY = y + tmpYMove
            idx = rangeFinder(obstaclePos, [tmpX,tmpY])
            FeatureVec1 = (tmpX - initPosGoal[0])**2 + (tmpY - initPosGoal[1])**2
            FeatureVec2 = 100000.0 / (0.0 + (tmpX - obstaclePos[idx][0])**2 + (tmpY - obstaclePos[idx][1])**2)
            Qs[i] = weightVec[0] * FeatureVec1 + weightVec[1] * FeatureVec2
#         print Qs
        action = np.argmax(Qs)
#         print action, '\r',
        [xMove, yMove] = takeAction(action)

        x = x + xMove
        y = y + yMove

        wallFlag = 0
        collisionFlag = 0
        pygame.draw.circle(screen, [100, 100, 255], [x,y], agentRadius, 0)

        initPosAgent = [x,y]

        if math.sqrt((x -  initPosGoal[0])**2 + (y - initPosGoal[1])**2) <= agentRadius:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
            rList.append(1)
        for i in range(0,obsNumber):
            obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
            obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
            if i == idx:
                pygame.draw.circle(screen, [255, 255, 50], obstaclePos[i], obstacleRadius, 0)
            else:
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) < 19:
                print("Collision!")
                collisionFlag = -1
                done = True
                rList.append(0)

        if done:
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
#             for i in range(0,obsNumber):
#                 obstaclePos[i][0] = int(150 + i % 15 * 100)
#                 obstaclePos[i][1] = int(yOffset + i / 15 * 100)
            for i in range(0,obsNumber):
                obstaclePos[i][0] = int(150 + i % 14 * 80)
                obstaclePos[i][1] = int(80 + i / 14 * 80)
            episodes.append(e)

        pygame.draw.circle(screen, [100,255,100], initPosGoal, 10, 2)
        pygame.display.flip()
        screen.fill([220,220,220])
    print("Percent of successful episodes: " + str(100.0 * sum(rList)/(e + 1)) + "%")

('Episode ', 0, 'Starts!')
Goal Reached!
Percent of successful episodes: 100.0%
('Episode ', 1, 'Starts!')
Collision!
Percent of successful episodes: 50.0%
('Episode ', 2, 'Starts!')
Collision!
Percent of successful episodes: 33.3333333333%
('Episode ', 3, 'Starts!')
Collision!
Percent of successful episodes: 25.0%
('Episode ', 4, 'Starts!')
Collision!
Percent of successful episodes: 20.0%
('Episode ', 5, 'Starts!')
Goal Reached!
Percent of successful episodes: 33.3333333333%
('Episode ', 6, 'Starts!')
Goal Reached!
Percent of successful episodes: 42.8571428571%
('Episode ', 7, 'Starts!')
Goal Reached!
Percent of successful episodes: 50.0%
('Episode ', 8, 'Starts!')
Collision!
Percent of successful episodes: 44.4444444444%
('Episode ', 9, 'Starts!')
Goal Reached!
Percent of successful episodes: 50.0%
('Episode ', 10, 'Starts!')
Goal Reached!
Percent of successful episodes: 54.5454545455%
('Episode ', 11, 'Starts!')
Goal Reached!
Percent of successful episodes: 58.3333333333%
('Episode 

Goal Reached!
Percent of successful episodes: 63.5416666667%
('Episode ', 96, 'Starts!')
Collision!
Percent of successful episodes: 62.8865979381%
('Episode ', 97, 'Starts!')
Goal Reached!
Percent of successful episodes: 63.2653061224%
('Episode ', 98, 'Starts!')
Collision!
Percent of successful episodes: 62.6262626263%
('Episode ', 99, 'Starts!')
Collision!
Percent of successful episodes: 62.0%
('Episode ', 100, 'Starts!')
Goal Reached!
Percent of successful episodes: 62.3762376238%
('Episode ', 101, 'Starts!')
Goal Reached!
Percent of successful episodes: 62.7450980392%
('Episode ', 102, 'Starts!')
Goal Reached!
Percent of successful episodes: 63.1067961165%
('Episode ', 103, 'Starts!')
Goal Reached!
Percent of successful episodes: 63.4615384615%
('Episode ', 104, 'Starts!')
Goal Reached!
Percent of successful episodes: 63.8095238095%
('Episode ', 105, 'Starts!')
Goal Reached!
Percent of successful episodes: 64.1509433962%
('Episode ', 106, 'Starts!')
Goal Reached!
Percent of success

Goal Reached!
Percent of successful episodes: 65.0793650794%
('Episode ', 189, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.2631578947%
('Episode ', 190, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.445026178%
('Episode ', 191, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.625%
('Episode ', 192, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.8031088083%
('Episode ', 193, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.9793814433%
('Episode ', 194, 'Starts!')
Goal Reached!
Percent of successful episodes: 66.1538461538%
('Episode ', 195, 'Starts!')
Collision!
Collision!
Percent of successful episodes: 65.8163265306%
('Episode ', 196, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.9898477157%
('Episode ', 197, 'Starts!')
Collision!
Percent of successful episodes: 65.6565656566%
('Episode ', 198, 'Starts!')
Goal Reached!
Percent of successful episodes: 65.8291457286%
('Episode ', 199, 'Starts!')
Goal Reached!

In [7]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

NameError: name 'num_episodes' is not defined